In [1]:
import numpy as np
import time
import pickle
import os
import copy
import argparse
import visualization_syntdata
from models import GFA_DiagonalNoiseModel, GFA_OriginalModel
from utils import GFAtools

In [2]:
def get_args():
    parser = argparse.ArgumentParser(description="GFA with two data sources")
    parser.add_argument("--scenario", nargs='?', default='incomplete', type=str,
                        help='Data scenario (complete or incomplete)')
    parser.add_argument("--noise", nargs='?', default='diagonal', type=str,
                        help='Noise assumption for GFA models (diagonal or spherical)')
    parser.add_argument("--num-sources", nargs='?', default=3, type=int,
                        help='Number of data sources')
    parser.add_argument("--K", nargs='?', default=6, type=int,
                        help='number of components to initialised the model')
    parser.add_argument("--impMedian", nargs='?', default=True, type=bool,
                        help='(not) impute median')
    
    args = parser.parse_args("")                                                               
    return args	 

In [3]:
# Get arguments 
args = get_args() 
# Define parameters to generate incomplete data sets
infoMiss = {'perc': [20,20], #percentage of missing data 
        'type': ['rows','random'], #type of missing data 
        'ds': [1,2]} #data sources that will have missing values        
    
# Make directory to save the results of the experiments         
res_dir = f'results/{args.num_sources}dsources/GFA_{args.noise}/{args.K}comps/{args.scenario}'
print(res_dir)
if not os.path.exists(res_dir):
        os.makedirs(res_dir)    

results/3dsources/GFA_diagonal/6comps/incomplete


In [4]:
def get_data_3g(args, infoMiss=None):

    """ 
    Generate synthetic data with 3 data sources.

    Parameters
    ----------
    args : local namespace 
        Arguments selected to run the model.

    infoMiss : dict | None, optional.
        Parameters selected to generate data with missing values.  

    Returns
    -------
    data : dict
        Training and test data as well as model parameters used 
        to generate the data.
    
    """
    Ntrain = 400; Ntest = 100
    N = Ntrain + Ntest #  total number of samples
    M = args.num_sources  #number of data sources
    d = np.array([50, 30, 20]) #number of dimensios in each data source
    true_K = 4  # true latent components
    # Specify Z manually
    Z = np.zeros((N, true_K))
    for i in range(0, N):
        Z[i,0] = np.sin((i+1)/(N/20))
        Z[i,1] = np.cos((i+1)/(N/20))
        Z[i,2] = 2 * ((i+1)/N-0.5)    
    Z[:,3] = np.random.normal(0, 1, N)          
    # Specify noise precisions manually
    tau = [[] for _ in range(d.size)]
    tau[0] = 5 * np.ones((1,d[0]))[0] 
    tau[1] = 10 * np.ones((1,d[1]))[0]
    tau[2] = 8 * np.ones((1,d[2]))[0]
    # Specify alphas manually
    alpha = np.zeros((M, true_K))
    alpha[0,:] = np.array([1,1,1e6,1])
    alpha[1,:] = np.array([1,1,1,1e6]) 
    alpha[2,:] = np.array([1,1e6,1,1e6]) 
    
    #W and X
    W = [[] for _ in range(d.size)]
    X_train = [[] for _ in range(d.size)]
    X_test = [[] for _ in range(d.size)]
    for i in range(0, d.size):
        W[i] = np.zeros((d[i], true_K))
        for t in range(0, true_K):
            #generate W from p(W|alpha)
            W[i][:,t] = np.random.normal(0, 1/np.sqrt(alpha[i,t]), d[i])
        X = np.zeros((N, d[i]))
        for j in range(0, d[i]):
            #generate X from the generative model
            X[:,j] = np.dot(Z,W[i][j,:].T) + \
            np.random.normal(0, 1/np.sqrt(tau[i][j]), N*1)    
        # Get training and test data
        X_train[i] = X[0:Ntrain,:] #Training data
        X_test[i] = X[Ntrain:N,:] #Test data
    #latent variables for training the model    
    Z = Z[0:Ntrain,:]

    # Generate incomplete training data
    if args.scenario == 'incomplete':
        X_train, missing_Xtrue = generate_missdata(X_train, infoMiss)
    
    # Store data and model parameters            
    data = {'X_tr': X_train, 'X_te': X_test, 'W': W, 'Z': Z, 'tau': tau, 'alpha': alpha, 'true_K': true_K}
    if args.scenario == 'incomplete':
        data.update({'trueX_miss': missing_Xtrue}) 
    return data 

def generate_missdata(X_train, infoMiss):
    """ 
    Generate missing data in the training data.

    Parameters
    ----------
    X_train : list 
        List of arrays containing the data matrix of each data 
        source.

    infoMiss : dict 
        Parameters selected to generate data with missing values.  

    Returns
    -------
    X_miss : list 
        List of arrays containing the training data. The data 
        sources specified in infoMiss will have missing values.

    missing_Xtrue : list 
        List of arrays containing the true values removed from the
        data sources selected in infoMiss.     
    
    """
    missing_Xtrue = [[] for _ in range(len(infoMiss['ds']))]
    for i in range(len(infoMiss['ds'])):
        g_miss = infoMiss['ds'][i]-1  
        if 'random' in infoMiss['type'][i]: 
            #remove entries randomly
            missing_val =  np.random.choice([0, 1], 
                        size=(X_train[g_miss].shape[0],X_train[g_miss].shape[1]), 
                        p=[1-infoMiss['perc'][i-1]/100, infoMiss['perc'][i-1]/100])
            mask_miss =  np.ma.array(X_train[g_miss], mask = missing_val).mask
            missing_Xtrue[i] = np.where(missing_val==1, X_train[g_miss],0)
            X_train[g_miss][mask_miss] = 'NaN'
        elif 'rows' in infoMiss['type'][i]: 
            #remove rows randomly
            Ntrain = X_train[g_miss].shape[0]
            missing_Xtrue[i] = np.zeros((Ntrain, X_train[g_miss].shape[1]))
            n_rows = int(infoMiss['perc'][i-1]/100 * Ntrain)
            shuf_samples = np.arange(Ntrain)
            np.random.shuffle(shuf_samples)
            missing_Xtrue[i][shuf_samples[0:n_rows],:] = X_train[g_miss][shuf_samples[0:n_rows],:]
            X_train[g_miss][shuf_samples[0:n_rows],:] = 'NaN'
        X_miss = X_train
    return X_miss, missing_Xtrue 


In [5]:
# Generate data
if args.scenario == 'complete':
    data = get_data_3g(args)
else:
    data = get_data_3g(args, infoMiss)

#save file with generated data
data_file = f'{res_dir}/[1]Data.dictionary'
with open(data_file, 'wb') as parameters:
        pickle.dump(data, parameters)           

In [6]:
X_tr = data['X_tr'] #get training data
# Initialise the model
if 'diagonal' in args.noise:    
    GFAmodel = GFA_DiagonalNoiseModel(X_tr, args)
else:
    assert args.scenario == 'complete'
    GFAmodel = GFA_OriginalModel(X_tr, args) 
    
#Fit the model
time_start = time.process_time()
GFAmodel.fit(X_tr)
GFAmodel.time_elapsed = time.process_time() - time_start
print(f'Computational time: {float("{:.2f}".format(GFAmodel.time_elapsed))}s')

ELBO (1st value): -259834.4572343341
ELBO (last value): -25181.67063802154
Number of iterations: 28
Computational time: 36.33s


In [7]:
# Predictions (Predict source 3 from source 1 and 2) 
# Compute mean squared error (MSE)
obs_ds = np.array([1, 1, 0]) #data source 1 and 2 were observed
gpred = np.where(obs_ds == 0)[0][0] #get the non-observed data source
X_test = data['X_te']
X_pred = GFAtools(X_test, GFAmodel).PredictDSources(obs_ds, args.noise)

#Compute MSE 
GFAmodel.MSE = np.mean((X_test[gpred] - X_pred[0]) ** 2)
print(f'MSE: {float("{:.2f}".format(GFAmodel.MSE))}')

# Compute MSE - chance level (MSE between test values and train means)
Tr_means = np.ones((X_test[gpred].shape[0], X_test[gpred].shape[1])) * \
    np.nanmean(data['X_tr'][gpred], axis=0)           
GFAmodel.MSE_chlev = np.mean((X_test[gpred] - Tr_means) ** 2)
print(f'MSE (chance level): {float("{:.2f}".format(GFAmodel.MSE_chlev))}')

# Save file containing model outputs and predictions
res_file = f'{res_dir}/[1]ModelOutput.dictionary'
with open(res_file, 'wb') as parameters:
    pickle.dump(GFAmodel, parameters)

MSE: 0.13
MSE (chance level): 1.34


In [8]:
if args.impMedian: 
    assert args.scenario == 'incomplete'
    X_impmed = copy.deepcopy(data['X_tr'])
    g_miss = np.array(infoMiss['ds']) - 1 #data source with missing values  
    # Impute median before training the model
    for i in range(g_miss.size):
        for j in range(data['X_tr'][g_miss[i]].shape[1]):
            Xtrain_j = data['X_tr'][g_miss[i]][:,j]
            X_impmed[g_miss[i]][np.isnan(X_impmed[g_miss[i]][:,j]),j] = np.nanmedian(Xtrain_j)

    # Initialise the model
    GFAmodel_median = GFA_DiagonalNoiseModel(X_impmed, args, imputation=True)
    # Fit the model
    time_start = time.process_time()
    GFAmodel_median.fit(X_impmed)
    GFAmodel_median.time_elapsed = time.process_time() - time_start
    print(f'Computational time: {float("{:.2f}".format(GFAmodel_median.time_elapsed))}s')

    # Predictions (Predict source 3 from source 1 and 2) 
    obs_ds = np.array([1, 1, 0]) #data source 1 and 2 were observed
    gpred = np.where(obs_ds == 0)[0][0] #get the non-observed data source
    X_test = data['X_te']
    X_pred = GFAtools(X_test, GFAmodel_median).PredictDSources(obs_ds, args.noise)
    GFAmodel_median.MSE = np.mean((X_test[gpred] - X_pred[0]) ** 2) 

    # Save file containing model outputs and predictions
    res_med_file = f'{res_dir}/[1]ModelOutput_median.dictionary'
    with open(res_med_file, 'wb') as parameters:
        pickle.dump(GFAmodel_median, parameters)

ELBO (1st value): -340578.60059115576
ELBO (last value): -32492.427570659915
Number of iterations: 53
Computational time: 3.85s


In [9]:
# Import auxiliary functions for visualization
from visualization_syntdata import hinton_diag, match_factors, plot_loadings, plot_Z
import matplotlib.pyplot as plt

model_1 = GFAmodel #model without imputation
model_2 = GFAmodel_median #model with median imputation

#Concatenate (true and estimated) loadings and alphas across data sources
#model_1
W_est_1 = np.zeros((np.sum(model_1.d),model_1.k))
alphas_est_1 = np.zeros((model_1.k, args.num_sources))
#model_2
W_est_2 = np.zeros((np.sum(model_2.d),model_2.k))
alphas_est_2 = np.zeros((model_2.k, args.num_sources))
#true parameters
W_true = np.zeros((np.sum(model_1.d),data['true_K']))
alphas_true = np.zeros((data['true_K'], args.num_sources))
d = 0
for m in range(args.num_sources):
    Dm = model_1.d[m]
    alphas_true[:,m] = data['alpha'][m]
    W_true[d:d+Dm,:] = data['W'][m]
    #model_1
    alphas_est_1[:,m] = model_1.E_alpha[m]
    W_est_1[d:d+Dm,:] = model_1.means_w[m]
    #model_2
    alphas_est_2[:,m] = model_2.E_alpha[m]
    W_est_2[d:d+Dm,:] = model_2.means_w[m]
    d += Dm  

In [10]:
# Plot loading matrices
#------------------------------------------------------------------
#plot true Ws
W_path = f'{res_dir}/[1]W_true.png'
plot_loadings(W_true, model_1.d, W_path) 

#plot estimated Ws - model 1
if model_1.k == data['true_K']:
    #match true and estimated components
    W_est_1, sim_factors_1, flip_1 = match_factors(W_est_1, W_true)
W_path_1 = f'{res_dir}/[1]W_est.png' 
plot_loadings(W_est_1, model_1.d, W_path_1)

#plot estimated Ws - model 2
if model_2.k == data['true_K']:
    #match true and estimated components
    W_est_2, sim_factors_2, flip_1 = match_factors(W_est_2, W_true)
W_path_2 = f'{res_dir}/[1]W_est_median.png' 
plot_loadings(W_est_2, model_2.d, W_path_2)

In [11]:
# Plot latent variables
#------------------------------------------------------------------------------------ 
#plot true latent variables 
Z_path = f'{res_dir}/[1]Z_true.png'    
plot_Z(data['Z'], Z_path)

#plot estimated latent variables - model 1
Z_path_1 = f'{res_dir}/[1]Z_est.png'
if model_1.k == data['true_K']:
    plot_Z(model_1.means_z, Z_path_1, match=True, s_comps=sim_factors_1, flip=flip_1)
else:     
    plot_Z(model_1.means_z, Z_path_1)    
    
#plot estimated latent variables - model 2
Z_path_2 = f'{res_dir}/[1]Z_est_median.png'
if model_2.k == data['true_K']:
    plot_Z(model_2.means_z, Z_path_2, match=True, s_comps=sim_factors_2, flip=flip_2)
else:     
    plot_Z(model_2.means_z, Z_path_2) 

In [12]:
# Plot alphas
#---------------------------------------------------------------------------
#plot true alphas
alphas_path = f'{res_dir}/[1]alphas_true.png'
hinton_diag(np.negative(alphas_true.T), alphas_path)     

#plot estimated alphas - model 1
alphas_path_1 = f'{res_dir}/[1]alphas_est.png'
if model_1.k == data['true_K']:
    hinton_diag(np.negative(alphas_est_1[sim_factors_1,:].T), alphas_path_1) 
else:
    hinton_diag(np.negative(alphas_est_1.T), alphas_path_1)

#plot estimated alphas - model 2
alphas_path_2 = f'{res_dir}/[1]alphas_est_median.png'
if model_2.k == data['true_K']:
    hinton_diag(np.negative(alphas_est_2[sim_factors_1,:].T), alphas_path_2) 
else:
    hinton_diag(np.negative(alphas_est_2.T), alphas_path_2)

In [13]:
# Plot ELBO - model 1
L_path_1 = f'{res_dir}/[1]ELBO.png'    
plt.figure(figsize=(5, 4))
plt.plot(model_1.L[1:])
plt.savefig(L_path_1)
plt.close() 

# Plot ELBO - model 2
L_path_2 = f'{res_dir}/[1]ELBO_median.png'
plt.figure(figsize=(5, 4))
plt.plot(model_2.L[1:])
plt.savefig(L_path_2)
plt.close()

In [21]:
import shutil
shutil.rmtree(res_dir, ignore_errors=True)

%run analysis_syntdata.py --num-sources=3

------------
Run: 1
Generating data---------
Data generated!
Running the model---------
ELBO (1st value): -323217.3283304632
ELBO (last value): -25385.50967991297
Number of iterations: 27
Computational time: 35.38s
Run Model after imp. median----------
ELBO (1st value): -304138.4595459776
ELBO (last value): -31616.140424463214
Number of iterations: 56
Computational time: 3.52s
------------
Run: 2
Generating data---------
Data generated!
Running the model---------
ELBO (1st value): -271729.49271066126
ELBO (last value): -24693.922939273765
Number of iterations: 31
Computational time: 38.85s
Run Model after imp. median----------
ELBO (1st value): -237134.16724238254
ELBO (last value): -31049.386928649452
Number of iterations: 15
Computational time: 1.62s
------------
Run: 3
Generating data---------
Data generated!
Running the model---------
ELBO (1st value): -331010.6127526672
ELBO (last value): -25369.631880315363
Number of iterations: 31
Computational time: 38.45s
Run Model after imp. 